# Training a Machine Learning model (Random Forest)

# Cài đặt thư viện cần thiết

In [ ]:
import numpy as np
import pandas as pd
from pprint import pprint
from numba import jit, prange
from numba import config
config.THREADING_LAYER = 'omp'

# Tìm hiểu data
## Input

* Link Dataset http://archive.ics.uci.edu/ml/datasets/connectionist+bench+(sonar,+mines+vs.+rocks)
* Tệp "sonar.mines" chứa 111 mẫu thu được bằng cách bật tín hiệu sonar ra khỏi một hình trụ kim loại ở nhiều góc độ khác nhau và trong các điều kiện khác nhau. 
* Tệp "sonar.rocks" chứa 97 mẫu thu được từ đá trong các điều kiện tương tự. Tín hiệu sonar truyền đi là một tiếng kêu được điều chỉnh tần số, tăng tần số. 
* Tập dữ liệu chứa các tín hiệu thu được từ nhiều góc độ khía cạnh khác nhau, trải dài 90 độ đối với hình trụ và 180 độ đối với đá.
* Mỗi mẫu là một tập hợp gồm 60 số trong phạm vi từ 0,0 đến 1,0. Mỗi số đại diện cho năng lượng trong một dải tần số cụ thể, được tích hợp trong một khoảng thời gian nhất định. 
* Khẩu độ tích hợp cho các tần số cao hơn xảy ra muộn hơn trong thời gian, vì các tần số này được truyền sau đó trong tiếng kêu.
* Nhãn liên kết với mỗi bản ghi chứa chữ cái "R" nếu đối tượng là một tảng đá và "M" nếu đó là mỏ (hình trụ kim loại).
![](https://i.imgur.com/zaH8WGi.png)

## Ouput
* Là model sau khi được train

##Source
* Bộ dữ liệu được đóng góp vào việc thu thập điểm chuẩn bởi Terry Sejnowski, hiện đang làm việc tại Viện Salk và Đại học California tại San Deigo. 
* Bộ dữ liệu được phát triển với sự hợp tác của R. Paul Gorman của Trung tâm Công nghệ Hàng không Vũ trụ Allied-Signal.


In [ ]:
data = pd.read_csv('/content/sonar.all-data.csv')
dataset = data.values
data

,0.02,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.018,0.0084,0.009,0.0032,0
0,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,0
1,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,0
2,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,0
3,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,0
4,0.0286,0.0453,0.0277,0.0174,0.0384,0.0990,0.1201,0.1833,0.2105,0.3039,...,0.0045,0.0014,0.0038,0.0013,0.0089,0.0057,0.0027,0.0051,0.0062,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,0.0187,0.0346,0.0168,0.0177,0.0393,0.1630,0.2028,0.1694,0.2328,0.2684,...,0.0116,0.0098,0.0199,0.0033,0.0101,0.0065,0.0115,0.0193,0.0157,1
203,0.0323,0.0101,0.0298,0.0564,0.0760,0.0958,0.0990,0.1018,0.1030,0.2154,...,0.0061,0.0093,0.0135,0.0063,0.0063,0.0034,0.0032,0.0062,0.0067,1
204,0.0522,0.0437,0.0180,0.0292,0.0351,0.1171,0.1257,0.1178,0.1258,0.2529,...,0.0160,0.0029,0.0051,0.0062,0.0089,0.0140,0.0138,0.0077,0.0031,1
205,0.0303,0.0353,0.0490,0.0608,0.0167,0.1354,0.1465,0.1123,0.1945,0.2354,...,0.0086,0.0046,0.0126,0.0036,0.0035,0.0034,0.0079,0.0036,0.0048,1


#Giới thiệu về Random Forest
* Thuật toán Random Forest sẽ xây dựng nhiều cây quyết định bằng thuật toán Decision Tree
* Tuy nhiên mỗi cây quyết định sẽ khác nhau (có yếu tố random). Sau đó kết quả dự đoán được tổng hợp từ các cây quyết định
* Decision Tree là tên đại diện cho một nhóm thuật toán phát triển dựa trên Cây quyết định. Ở đó, mỗi Node của cây sẽ là các thuộc tính, và các nhánh là giá trị lựa chọn của thuộc tính đó. Bằng cách đi theo các giá trị thuộc tính trên cây,
Cây quyết định sẽ cho ta biết giá trị dự đoán.


##Ưu điểm

* Random Forest algorithm có thể sử dụng cho cả bài toán Classification và Regression
* Khi Forest có nhiều cây hơn, chúng ta có thể tránh được việc Overfitting với tập dữ liệu

## Khuyết điểm
* Random forests chậm tạo dự đoán bởi vì nó có nhiều cây quyết định. 
* Bất cứ khi nào nó đưa ra dự đoán, tất cả các cây trong rừng phải đưa ra dự đoán cho cùng một đầu vào cho trước và sau đó thực hiện bỏ phiếu trên đó. Toàn bộ quá trình này tốn thời gian. 

##Ứng dụng
* Thuật toán Random Forest được sử dụng trong nhiều lĩnh vực khác nhau, chẳng hạn như ngân hàng, thị trường tài chính, y học, thương mại điện tử. 
*Ví dụ, trong lĩnh vực ngân hàng, nó được sử dụng để phát hiện những khách hàng sẽ sử dụng dịch vụ ngân hàng thường xuyên hơn những người khác và trả nợ đúng hạn. Trong lĩnh vực này, nó cũng được sử dụng để phát hiện hành vi gian lận của khách hàng muốn gây hại cho ngân hàng. 
* Trong lĩnh vực tài chính, nó được sử dụng để xác định hiệu suất trong tương lai của một cổ phiếu. 
* Trong chăm sóc sức khỏe, nó được sử dụng để xác định sự kết hợp chính xác của các thành phần trong y học và nó cũng được sử dụng để phân tích bệnh sử của bệnh nhân để xác định bệnh tật. 
* Trong thương mại điện tử, Random Forest được sử dụng để xác định xem khách hàng có thích sản phẩm hay không.

## Cách hoạt động
* Chọn các mẫu ngẫu nhiên từ tập dữ liệu đã cho.
* Thiết lập cây quyết định cho từng mẫu và nhận kết quả dự đoán từ mỗi quyết định cây.
* Hãy bỏ phiếu cho mỗi kết quả dự đoán.
* Chọn kết quả được dự đoán nhiều nhất là dự đoán cuối cùng.
![](https://i.imgur.com/O5TQ5K6.png)


## Mã giả tạo ra nhiều cây 
* Chọn ngẫu nhiên “k” features từ tập “m” features. Chọn ngẫu nhiên số dòng dữ liệu từ dataset.
Để ý k << m

* Từ tập “k” features, tính toán ra node “d” là tốt nhất cho Node phân loại.

* Chia các node con theo node tốt nhất vừa tìm được

* Dùng thuật toán Decision Tree tạo cây cho Node tốt nhất.

* Lặp lại bước 1-4 để tạo ra “n” cây

#Code tuần tự

In [ ]:
@jit(parallel=True, cache=True)
def gini(df):
        x_unique,x_count = np.unique(df[:,-1], return_counts=True)
        x_prob = x_count/np.sum(x_count)
        x_gini = 1 - (np.dot(x_prob,x_prob))
        return x_gini

In [ ]:
## function picks the best value to split the real valued attribute 
def pick_the_value(df,n):
        n_best = []
        loss = 1000 # loss would not be more than 1 as gini is atmost 0.5
        unique = np.sort(np.unique(df[:,n]))
        split_values = [((unique[index]+unique[index-1])/2) for index in range(1,len(unique))]
        if len(split_values) > 100 :
            strip = list(range(1,100))
            steps = np.ptp(df[:,n])/100
            mini = np.min(df[:,n])
            split_values = mini+(np.asarray(strip)*steps)

        for i in split_values:
            above = df[df[:,n] > i]
            below = df[df[:,n] <= i] 
            a_gini = gini(above)
            b_gini = gini(below)
            a_len = len(above)
            b_len = len(below)
            current_loss = ((a_len/(a_len+b_len))*a_gini) + ((b_len/(a_len+b_len))*b_gini)
            
            if current_loss < loss:
                loss = current_loss
                n_best = [i,loss,above,below,n]
        return n_best

In [ ]:
## picks the best node(attribute) to split
def decision_node(df):
        node_split = []
        par_loss = 1000
        col_no = df.shape[1] - 1
        par_list = np.random.choice(col_no,int(np.floor(np.sqrt(col_no))),replace=False)
        for i in par_list:
            i_best = pick_the_value(df,i)

            if len(i_best)!=0 :
                if i_best[1] < par_loss:
                    par_loss = i_best[1]
                    node_split = i_best
        return node_split

In [ ]:
## displaying the decision tree with tree pruning option (we can decide till which height the tree can go before stop, it decides the label based on the majority)
def decisiontree(df):#,c = 0, m = 3):
        x_unique = np.unique(df[:,-1])
        if len(x_unique) == 1:# or c == m:
            return x_unique[0]
        else :
            # c+=1
            split = decision_node(df)
            col_no = split[4]
            value = split[0]
            df_above = split[2]
            df_below = split[3]
            condition = "{} <= {}".format(col_no,value)
            decision_tree = {condition : []}
            true = decisiontree(df_below)#,c,m)
            false = decisiontree(df_above)#,c,m)

            decision_tree[condition].append(true)
            decision_tree[condition].append(false)

            return decision_tree

In [ ]:
## bootstrapping the datasets to generate multiple decision trees 
def bootstrap_decision_tree(df):
        list_bootstrapped_tree = []
        n = 15 ## number of bootstrapped datasets need to be created
        for i in range(n):
            index = np.random.choice(len(df),len(df),replace=True)
            list_bootstrapped_tree.append(decisiontree(df[index]))
        return list_bootstrapped_tree

In [ ]:
## test sample is being passed through the decision tree and label is assigned
def classify(test_sample,tree):
        condition = list(tree.keys())[0]
        column_number = condition.split()[0]
        value = condition.split()[2]
        if test_sample[int(column_number)] <= float(value):
            label = tree[condition][0]
        else :
            label = tree[condition][1]
        if type(label) == dict:
            sub_tree = label
            return classify(test_sample,sub_tree)
        else :
            return label

In [ ]:
## label for a test sample is picked based on majority among the labels generated from bootstrapped decision trees for that test sample
def predict(x_test,list_bootstrapped_tree):
        y_pred = []#np.asarray([])
        for test in x_test:
            label_list = []
            for i in list_bootstrapped_tree:
                label_list.append(classify(test,i))
            unique_labels,count = np.unique(label_list,return_counts=True)
            y_pred.append(unique_labels[np.argmax(count)])
        return y_pred

In [ ]:
@jit(parallel=True, cache=True)
def RandomForest(X_train,Y_train,X_test):   
    Y_train = np.asarray([Y_train])
    Y_train = np.swapaxes(Y_train,0,1)
    df = np.append(X_train,Y_train,axis=1)
    list_bootstrapped_tree = bootstrap_decision_tree(df)
    y_pred = predict(X_test,list_bootstrapped_tree)
    return y_pred

In [ ]:
@jit(parallel=True, cache=True)
def Accuracy(y_true,y_pred):
    nppred = np.asarray(y_pred)
    c = y_true - nppred
    misclass = np.count_nonzero(c)
    test_len = len(y_true)
    accuracy = 1 - (misclass/test_len)
    return accuracy

In [ ]:
# np.random.seed(4563)
data = pd.read_csv("/content/sonar.all-data.csv", header = None , skiprows = 1)
dataframe = pd.DataFrame()
dataframe = pd.DataFrame(data)
Y = dataframe.pop(dataframe.columns[-1])
ind = np.random.choice(len(dataframe),len(dataframe)*8//10,replace=False)
df = np.asarray(dataframe)
label = np.asarray(Y)
X_train = df[ind]
X_test = np.delete(df,ind,axis = 0)
Y_train = label[ind]
Y_test = np.delete(label,ind,axis = 0)
# train_len = len(x_train)
# test_len = len(x_test)
# col_len = x_train.shape[1]
x_train_norm = (X_train - np.mean(X_train,axis = 0)[np.newaxis,:])/np.std(X_train,axis=0)[np.newaxis,:]
x_test_norm = (X_test - np.mean(X_test,axis = 0)[np.newaxis,:])/np.std(X_test,axis=0)[np.newaxis,:]

In [ ]:
y_pred_rf = RandomForest(X_train,Y_train,X_test)
print("Accuracy :",Accuracy(Y_test,y_pred_rf))

Accuracy : 0.7380952380952381
